In [28]:
import csv
import pandas as pd
tmp_catalog = '/home/jimmyli/announcement/'

data_x = pd.read_csv(filepath_or_buffer='/home/jimmyli/announcement/bulletinu.csv', sep = ','
                    )["公告类别"].values
data_y = pd.read_csv(filepath_or_buffer='/home/jimmyli/announcement/bulletinu.csv', sep = ','
                    )["公告主题"].values
Hlength = 833

f = open('/home/jimmyli/announcement/data.txt', 'w')

for i in range(1, Hlength):
    # print(data_x[i], data_y[i])
    f.write(str(data_x[i]))
    f.write('\t')
    f.write(str(data_y[i]))
    f.write('\n')
f.close()

In [29]:
"""
将未识别图片的正文提取到content.txt中
"""
import csv
from bs4 import BeautifulSoup
#打开文件，用with打开可以不用去特意关闭file了，python3不支持file()打开文件，只能用open()
with open("/home/jimmyli/announcement/content.txt", 'w') as w:
    with open("/home/jimmyli/bulletinu.csv","r",encoding="utf-8") as csvfile:
     #读取csv文件，返回的是迭代类型

    
        dictread=csv.DictReader(csvfile)
    
        for i ,row in enumerate(dictread):
            if(i<833):
            
                html = row['公告内容']
                bsObj= BeautifulSoup(html,"lxml")
                w.write(bsObj.get_text())
                w.write('\n')
"""
将正文中的空格去除存到content_copy.txt
"""
import os 

with open((os.path.join('content.txt')), 'r') as f:
    with open((os.path.join('content_copy.txt')), 'w') as w:
        data = f.readlines()
        for line in data:
            odom = line.split()
            tmp_str = "".join(odom)
            result = ' '.join(tmp_str.split())
            w.write(result)
            w.write('\n')
"""
将正文与标题结合生成data_full.txt
"""
tmp_catalog = '/home/jimmyli/announcement/'

data = []
content = []
with open(tmp_catalog + 'data_full.txt', 'w') as w:
    with open(tmp_catalog + 'data.txt', 'r') as t:
        with open(tmp_catalog + 'content_copy.txt', 'r') as f:
            for line in t:
                line = line.rstrip('\n')
                data.append(line)
            content = f.readlines()
            for i in range(len(data)):
                w.write(data[i] + ' ' + content[i])


In [42]:
'''分词,存放在full_token.txt'''
import multiprocessing
import jieba

tmp_catalog = '/home/jimmyli/announcement/'

def tokenFile(file_path, write_path):

    with open(write_path, 'w') as w:
        with open(file_path, 'r') as f:
            for line in f:
                token_sen = (" ".join(jieba.cut(line.split('\t')[1])))               
                w.write(line.split('\t')[0] + '\t' + token_sen)
    print(file_path + ' has been token and token_file_name is ' + write_path)

tokenFile(tmp_catalog+'data_full.txt', tmp_catalog+'full_token.txt')


/home/jimmyli/announcement/data_full.txt has been token and token_file_name is /home/jimmyli/announcement/full_token.txt


In [43]:
def splitData(file_path, train_write_path, test_write_path):
    
    with open(train_write_path, 'w') as w:        
        with open(test_write_path, 'w') as t:           
            with open(file_path, 'r') as f:
                flist = f.readlines()
                for i in range(len(flist)):
                    if i % 5 == 0:
                        t.write(flist[i])
                    else:
                        w.write(flist[i])
                    
splitData(tmp_catalog+'full_token.txt', tmp_catalog+'full_train.txt',
         tmp_catalog+'full_test.txt')    

In [45]:
def constructDataset(path):
    """
    path: file path
    rtype: lable_list and corpus_list
    """
    label_list = []
    corpus_list = []
    with open(path, 'r') as p:
        for line in p:
            label_list.append(line.split('\t')[0])
            corpus_list.append(line.split('\t')[1])
    return label_list, corpus_list

file_path = ['full_test.txt', 'full_train.txt']
test_label, test_set = constructDataset(tmp_catalog + file_path[0])   #167
train_label, train_set = constructDataset(tmp_catalog + file_path[1]) #664
print(len(train_label))

664


In [46]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

corpus_set = train_set + test_set
print("length of corpus is:" + str(len(corpus_set)))

vectorizer = CountVectorizer(min_df=1e-5) # drop df < le-5,去低频词

transfomer = TfidfTransformer()

tfidf = transfomer.fit_transform(vectorizer.fit_transform(corpus_set))

words = vectorizer.get_feature_names()

print("how many words: {0}".format(len(words)))

print("tf-idf shape: ({0}, {1})".format(tfidf.shape[0], tfidf.shape[1]))

length of corpus is:831
how many words: 3106
tf-idf shape: (831, 3106)


In [47]:
from sklearn import preprocessing

# encode label
corpus_label = train_label + test_label
encoder = preprocessing.LabelEncoder()
corpus_encode_label = encoder.fit_transform(corpus_label)
train_label = corpus_encode_label[:664]
test_label = corpus_encode_label[664:]
# get tf-idf dataset
train_set = tfidf[:664]
test_set = tfidf[664:]

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# LogisticRegression classiy model
lr_model = LogisticRegression()
lr_model.fit(train_set, train_label)
y_pred = lr_model.predict(test_set)
print (classification_report(test_label, y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         9
          1       1.00      0.11      0.20         9
          2       0.56      0.50      0.53        20
          3       0.91      0.56      0.69        18
          4       0.50      0.22      0.31        18
          5       0.00      0.00      0.00         5
          6       0.50      0.36      0.42        14
          7       0.67      0.17      0.27        12
          9       0.80      0.50      0.62        16
         10       0.41      0.93      0.57        46

avg / total       0.55      0.50      0.45       167



/home/jimmyli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [49]:
encoder.classes_

array(['GMO', 'IT Bulletin', '人力资源', '公共事务', '环境卫生安全', '社团活动', '福委会',
       '设施服务', '财会税务', '资讯安全', '餐饮服务'], dtype='<U11')